In [ ]:
%help

In [3]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
import os
import json
import functools
import boto3
import numpy as np
import pandas as pd
from datetime import datetime
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

You are already connected to a glueetl session 4e61c2f1-ba10-4214-b815-b1b4fea6c335.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Current idle_timeout is 2880 minutes.
idle_timeout has been set to 2880 minutes.


You are already connected to a glueetl session 4e61c2f1-ba10-4214-b815-b1b4fea6c335.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 5.0


You are already connected to a glueetl session 4e61c2f1-ba10-4214-b815-b1b4fea6c335.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous worker type: G.1X
Setting new worker type to: G.1X


You are already connected to a glueetl session 4e61c2f1-ba10-4214-b815-b1b4fea6c335.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Previous number of workers: 5
Setting new number of workers to: 5



In [4]:
# Initialize the S3 client
s3_client = boto3.client('s3')

In [5]:
# Define your S3 bucket name and the file path within the bucket
bucket_name = 'cci-dig-aicoe-data-sb'
file_key = 'processed/healthscore/healthscore_json/healthscore_dashboard.json'

raw_file_path= 's3://cci-dig-aicoe-data-sb/processed/healthscore/healthscore_raw_file/HS_DataClean_Nov.csv'

In [6]:
spark_df = spark.read.option("header", "true").csv(raw_file_path)
spark_df.show()

+--------------------+---------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------+----------

In [7]:
df= spark_df.toPandas()

In [8]:
df.head()

           Metrics Operating System Type  ... 25-11-2024 26-11-2024
0        DR_Visits             Apple iOS  ...     116462     114579
1        DR_Visits        Google Android  ...      35820      35696
2  Unique Visitors             Apple iOS  ...     101365      99610
3  Unique Visitors        Google Android  ...      32319      32219
4       Page Views             Apple iOS  ...    1524040    1532116

[5 rows x 92 columns]


In [9]:
# Read the JSON file from S3
read_json = s3_client.get_object(Bucket=bucket_name, Key=file_key)

# Load the file content into the config_data variable
config_data = json.loads(read_json['Body'].read().decode('utf-8'))

# Print the data to verify
print(config_data)

{'features': [{'feature_name': 'OVER ALL', 'feature_seq_num': '1', 'metrics': [{'metrics': 'DR_Visits', 'display_names': 'Visits', 'metric_nature': 'Positive', 'metric_sequence_num': '1'}, {'metrics': 'Unique Visitors', 'display_names': 'Visitors', 'metric_nature': 'Positive', 'metric_sequence_num': '2'}, {'metrics': 'Page Views', 'display_names': 'Page Views', 'metric_nature': 'Positive', 'metric_sequence_num': '3'}, {'metrics': 'Average Time Spent on Page', 'display_names': 'Average Time Spent (mins)', 'metric_nature': 'Negative', 'metric_sequence_num': '4'}, {'metrics': 'DR_New Visitors', 'display_names': 'New Visitors', 'metric_nature': 'Positive', 'metric_sequence_num': '5'}, {'metrics': 'DR_Server Errors', 'display_names': 'Server Errors', 'metric_nature': 'Negative', 'metric_sequence_num': '6'}, {'metrics': 'DR_Overall Chat', 'display_names': 'Visits to Chat', 'metric_nature': 'Negative', 'metric_sequence_num': '7'}, {'metrics': 'DR_Message Us', 'display_names': 'Visits to Chat 

In [2]:
#df = pd.read_csv(r"D:\shalini\HS_DataClean_Nov.csv")

In [4]:
# Step 1: Read the config.json file
#with open("D:\\Healthscore Codes\\healthscore_dashboard.json", 'r') as file:
#    config_data = json.load(file)
#    print(config_data)

{'features': [{'feature_name': 'OVER ALL', 'feature_seq_num': '1', 'metrics': [{'metrics': 'DR_Visits', 'display_names': 'Visits', 'metric_nature': 'Positive', 'metric_sequence_num': '1'}, {'metrics': 'Unique Visitors', 'display_names': 'Visitors', 'metric_nature': 'Positive', 'metric_sequence_num': '2'}, {'metrics': 'Page Views', 'display_names': 'Page Views', 'metric_nature': 'Positive', 'metric_sequence_num': '3'}, {'metrics': 'Average Time Spent on Page', 'display_names': 'Average Time Spent (mins)', 'metric_nature': 'Negative', 'metric_sequence_num': '4'}, {'metrics': 'DR_New Visitors', 'display_names': 'New Visitors', 'metric_nature': 'Positive', 'metric_sequence_num': '5'}, {'metrics': 'DR_Server Errors', 'display_names': 'Server Errors', 'metric_nature': 'Negative', 'metric_sequence_num': '6'}, {'metrics': 'DR_Overall Chat', 'display_names': 'Visits to Chat', 'metric_nature': 'Negative', 'metric_sequence_num': '7'}, {'metrics': 'DR_Message Us', 'display_names': 'Visits to Chat 

In [10]:
# Function to parse the nested dictionary
def parse_data(data):
    feature_groups = {}
    feature_seqno_mapping = {}
    display_name_mapping = {}
    metric_nature_mapping = {}
    metric_sequence_number_mapping = {}

    # Loop through the features in the data
    for feature in data['features']:
        feature_name = feature['feature_name']
        feature_seq_num = feature['feature_seq_num']
        
        # Add feature to feature groups
        feature_groups[feature_name] = []
        
        # Add feature seq no mapping
        feature_seqno_mapping[feature_name] = feature_seq_num
        
        for metric in feature['metrics']:
            metric_name = metric['metrics']
            display_name = metric['display_names']
            metric_nature = metric['metric_nature']
            metric_seq_num = metric['metric_sequence_num']
            
            # Add display name mapping
            display_name_mapping[metric_name] = display_name
            
            # Add metric nature mapping
            metric_nature_mapping[display_name] = metric_nature
            
            # Add metric sequence number mapping
            metric_sequence_number_mapping[display_name] = metric_seq_num
            
            # Add the metric name to the respective feature group
            feature_groups[feature_name].append(metric_name)

    return feature_groups, feature_seqno_mapping, display_name_mapping, metric_nature_mapping, metric_sequence_number_mapping

In [11]:
# Parse the data and get the mappings
feature_groups, feature_seqno_mapping, display_name_mapping, metric_nature_mapping, metric_sequence_number_mapping = parse_data(config_data)

In [12]:
# Step 2: Extract config1 and config2 from the file
level_1_metrics = config_data['level_1_metrics']
level_2_metrics = config_data['level_2_metrics']
level_3_metrics = config_data['level_3_metrics']
level_4_metrics = config_data['level_4_metrics']
level_5_metrics = config_data['level_5_metrics']


In [13]:
# Print the results (optional)
print("Feature Groups =", feature_groups)
print("Feature Seqno Mapping =", feature_seqno_mapping)
print("Display Name Mapping =", display_name_mapping)
print("Metric Nature Mapping =", metric_nature_mapping)
print("Metric Sequence Number Mapping =", metric_sequence_number_mapping)
print("level_1_metrics = ", level_1_metrics)
print("level_2_metrics = ", level_2_metrics)
print("level_3_metrics = ", level_3_metrics)
print("level_4_metrics = ", level_4_metrics)
print("level_5_metrics = ", level_5_metrics)

Feature Groups = {'OVER ALL': ['DR_Visits', 'Unique Visitors', 'Page Views', 'Average Time Spent on Page', 'DR_New Visitors', 'DR_Server Errors', 'DR_Overall Chat', 'DR_Message Us', 'DR_Agent Chat', 'DR_Self Service Initiated', 'DR_CoxApp | Visits with Chat and No Self Service Initiated', 'DR_CoxApp | Visits with No Self Service and No Chat', 'DR_CoxApp | Visits not logged in', 'DR_continuing chat from new chat page'], 'LOG IN': ['DR_Login Page Visits', 'DR_Successful logins', 'DR_Auto Logins', 'DR_Clicks to MFA Verification', 'DR_Clicks on Forgot User ID', 'Clicks on Forgot Password', 'DR_Need to Register'], 'BIL LING': ['DR_Visits to Billing Home', 'DR_Billing Make a Payment', 'DR_Payment Submit', 'DR_Payments', 'DR_Client Errors on Make Payment', 'DR_Server Errors on Make Payment', 'DR_Promise to Pay', 'DR_Promise to Pay (Native)', 'DR_CoxApp | Billing - Additional time to pay (Link out)', 'DR_CoxApp | Billing - Future date payment (Link out)', 'DR_Billing - View Bill Details', 'DR_

In [14]:
df['display_names'] = df['Metrics'].map(display_name_mapping)

In [15]:
df['metric_nature'] = df['display_names'].map(metric_nature_mapping)

In [16]:
df['level_1'] = df['display_names'].apply(lambda x: x if x in level_1_metrics else None)
df['level_2'] = df['display_names'].apply(lambda x: x if x in level_2_metrics else None)
df['level_3'] = df['display_names'].apply(lambda x: x if x in level_3_metrics else None)
df['level_4'] = df['display_names'].apply(lambda x: x if x in level_4_metrics else None)
df['level_5'] = df['display_names'].apply(lambda x: x if x in level_5_metrics else None)

In [17]:
feature_column = 'Unknown'
df['Feature'] = feature_column
for group, metrics in feature_groups.items():
   df['Feature'] = np.where(df['Metrics'].isin(metrics), group, df['Feature'])

In [18]:
def move_row(df, from_idx, to_idx):
    row = df.iloc[from_idx]
    df = df.drop(from_idx).reset_index(drop=True)
    df = pd.concat([df.iloc[:to_idx], row.to_frame().T, df.iloc[to_idx:]]).reset_index(drop=True)
    return df

df = move_row(df, 50, 46)
df = move_row(df, 51, 47)
df = move_row(df, 52, 48)
df = move_row(df, 53, 49)
df = move_row(df, 164, 158)
df = move_row(df, 165, 159)
df = move_row(df, 180, 174)
df = move_row(df, 181, 175)
df = move_row(df, 218, 206)
df = move_row(df, 219, 207)
df = move_row(df, 220, 208)
df = move_row(df, 221, 209)
df = move_row(df, 280, 268)
df = move_row(df, 281, 269)
df = move_row(df, 282, 270)
df = move_row(df, 283, 271)

In [19]:
df['unique_identifier'] = df['Feature'].apply(lambda x: ''.join([word[0].upper() for word in x.split()]))

In [20]:
def generate_hierarchy_id_grouped_by_os(df):
    df['Hierarchy_ID'] = None

   
    df['Feature_Order'] = df.groupby('Operating System Type')['unique_identifier'].transform(lambda x: pd.Series(range(len(x)), index=x.index))

    grouped = df.groupby(['Operating System Type', 'unique_identifier'], group_keys=False)
    results = []

   
    for (os_name, unique_id), group in grouped:
        current_ids = {'level_1': 0, 'level_2': 0, 'level_3': 0, 'level_4': 0, 'level_5': 0}  

        group = group.sort_values('Feature_Order')  

        for index, row in group.iterrows():
            if pd.isna(row['display_names']):
                continue  

            if not pd.isna(row['level_1']):  
                current_ids['level_1'] += 1
                current_ids['level_2'] = 0  
                current_ids['level_3'] = 0  
                current_ids['level_4'] = 0  
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}"

            elif not pd.isna(row['level_2']):  
                current_ids['level_2'] += 1
                current_ids['level_3'] = 0  
                current_ids['level_4'] = 0  
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}"

            elif not pd.isna(row['level_3']):  
                current_ids['level_3'] += 1
                current_ids['level_4'] = 0  
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}.{current_ids['level_3']}"

            elif not pd.isna(row['level_4']):  
                current_ids['level_4'] += 1
                current_ids['level_5'] = 0  
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}.{current_ids['level_3']}.{current_ids['level_4']}"

            elif not pd.isna(row['level_5']):  
                current_ids['level_5'] += 1
                group.at[index, 'Hierarchy_ID'] = f"{os_name}_{unique_id}_{current_ids['level_1']}.{current_ids['level_2']}.{current_ids['level_3']}.{current_ids['level_4']}.{current_ids['level_5']}"

        results.append(group)

   
    df = pd.concat(results, ignore_index=True)

    
    #df = df[df['Display_Names'].notna()]

    
    cols = [col for col in df.columns if col != 'Hierarchy_ID' and col != 'Feature_Order'] + ['Hierarchy_ID']
    return df[cols]


df = generate_hierarchy_id_grouped_by_os(df)


In [21]:
df = generate_hierarchy_id_grouped_by_os(df)

def assign_parent_id(row):
    if not pd.isna(row['level_5']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_4']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_3']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_2']):  
        return f"{row['Hierarchy_ID'].rsplit('.', 1)[0]}"
    elif not pd.isna(row['level_1']): 
        return None
    else:
        return None


df['Parent_ID'] = df.apply(assign_parent_id, axis=1)


In [22]:
df['metric_id'] = range(1, len(df) + 1)

In [23]:
df['metric_sequence_num'] = df.groupby('Operating System Type').cumcount() + 1

In [24]:
df['metric_seqno'] = df['display_names'].map(metric_sequence_number_mapping)

In [25]:
def calculate_level_no(row):
   
    if pd.isna(row['Hierarchy_ID']) or pd.isna(row['Parent_ID']):
        return 1
    
   
    return row['Hierarchy_ID'].count('.') + 1


df['level_no'] = df.apply(calculate_level_no, axis=1)


In [26]:
feature_names = df['Feature'].unique()
feature_mapping = {feature_name: idx + 1 for idx, feature_name in enumerate(feature_names)}

df['feature_id'] = df['Feature'].map(feature_mapping)


In [27]:
# Create 'feature_seq_num' column based on the dictionary
df['feature_seq_num'] = df['Feature'].map(feature_seqno_mapping)

In [28]:
df['create_dt'] = pd.to_datetime('today').normalize().date()

In [29]:
df = df.rename(columns={'Parent_ID': 'parent_id_old'})

In [30]:
def calculate_parent_id(row, df):
    
    if pd.isna(row['Hierarchy_ID']):
        return None
   
    hierarchy_parts = row['Hierarchy_ID'].rsplit('.', 1)
    if len(hierarchy_parts) > 1:
       
        parent_hierarchy = hierarchy_parts[0]
        
        parent_row = df[df['Hierarchy_ID'] == parent_hierarchy]
        if not parent_row.empty:
            return parent_row['metric_id'].values[0]

    return None

df['parent_id'] = df.apply(calculate_parent_id, axis=1, df=df)


df.loc[df['Hierarchy_ID'].isna(), 'parent_id'] = None


In [31]:
df = df.rename(columns={
    'Metrics': 'metrics',
    'Operating System Type': 'operating_system_type',
    'Feature': 'feature_name',
    'Operating System Type': 'operating_system_type',
    'Feature': 'feature_name',
    'Operating System Type': 'operating_system_type',
    'Hierarchy_ID': 'hierarchy_id',
})

In [32]:
df.shape

(463, 111)


In [33]:
print(list(df.columns))

['metrics', 'operating_system_type', '29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024', '23-10-2024', '24-10-2024', '25-10-2024', '26-10-2024', '27-10-2024', '28-10-2024', '29-10-2024', '30-10-2024', '31-10-2024', '01-11-2024', '02-11-2024', '03-11-2024', '04-11-2024', '05-11-2024

In [34]:
first_table_columns = ['metric_id', 'metrics', 'display_names', 'operating_system_type', 'feature_name', 'unique_identifier', 'level_1', 'level_2', 'level_3', 'level_4','level_5', 'hierarchy_id', 'parent_id_old', 'metric_sequence_num', 'metric_seqno', 'level_no', 'feature_id', 'feature_seq_num', 'parent_id', 'metric_nature', 'create_dt' ]
first_table_df = df[first_table_columns]

In [9]:
#first_table_df.to_csv(r'D:\Healthsore Data\table_1_data_all_features.csv', index = False)

In [35]:
def is_date_column(col_name):
    date_formats = ['%d-%m-%Y']
    for fmt in date_formats:
        try:
            pd.to_datetime(col_name, format=fmt)
            return True
        except ValueError:
            continue  # Try the next format
    return False

# Filter columns with date format 'm/d/Y'
date_columns = [col for col in df.columns if is_date_column(col)]

In [36]:
second_table_columns1 = ['metric_id', 'create_dt', 'metrics', 'operating_system_type']
second_table_columns = second_table_columns1 + date_columns
second_table_df = df[second_table_columns]

In [37]:
filtered_date_columns = second_table_df.columns[4:].to_list()

In [38]:
id_vars = ['metric_id', 'create_dt']

In [39]:
second_table_df_melted = second_table_df.melt(id_vars = id_vars, value_vars = filtered_date_columns, var_name = 'Date', value_name = 'Value')

In [40]:
second_table_df_melted.shape

(41670, 4)


In [41]:
second_table_spark_df = spark.createDataFrame(second_table_df_melted)
second_table_spark_df.show()

+---------+----------+----------+-----+
|metric_id| create_dt|      Date|Value|
+---------+----------+----------+-----+
|        1|2025-02-10|29-08-2024|71742|
|        2|2025-02-10|29-08-2024|28280|
|        3|2025-02-10|29-08-2024|    1|
|        4|2025-02-10|29-08-2024|    1|
|        5|2025-02-10|29-08-2024|26212|
|        6|2025-02-10|29-08-2024|25656|
|        7|2025-02-10|29-08-2024|    0|
|        8|2025-02-10|29-08-2024| 3022|
|        9|2025-02-10|29-08-2024| 1969|
|       10|2025-02-10|29-08-2024|  227|
|       11|2025-02-10|29-08-2024| 6387|
|       12|2025-02-10|29-08-2024| 5111|
|       13|2025-02-10|29-08-2024| 2449|
|       14|2025-02-10|29-08-2024| 2675|
|       15|2025-02-10|29-08-2024| 2005|
|       16|2025-02-10|29-08-2024|  784|
|       17|2025-02-10|29-08-2024|  137|
|       18|2025-02-10|29-08-2024|  780|
|       19|2025-02-10|29-08-2024| 2236|
|       20|2025-02-10|29-08-2024| 3630|
+---------+----------+----------+-----+
only showing top 20 rows


In [42]:
second_table_spark_df.printSchema()

root
 |-- metric_id: long (nullable = true)
 |-- create_dt: date (nullable = true)
 |-- Date: string (nullable = true)
 |-- Value: string (nullable = true)


In [43]:
from pyspark.sql import functions as F

# Convert metric_id to integer, Date to date format, and Value to integer
second_table_spark_df_transformed = second_table_spark_df \
    .withColumn("metric_id", F.col("metric_id").cast("int")) \
    .withColumn("Date", F.to_date(F.col("Date"), "dd-MM-yyyy")) \
    .withColumn("Value", F.col("Value").cast("int"))

# Show the updated DataFrame to verify the changes
second_table_spark_df_transformed.show()


+---------+----------+----------+-----+
|metric_id| create_dt|      Date|Value|
+---------+----------+----------+-----+
|        1|2025-02-10|2024-08-29|71742|
|        2|2025-02-10|2024-08-29|28280|
|        3|2025-02-10|2024-08-29|    1|
|        4|2025-02-10|2024-08-29|    1|
|        5|2025-02-10|2024-08-29|26212|
|        6|2025-02-10|2024-08-29|25656|
|        7|2025-02-10|2024-08-29|    0|
|        8|2025-02-10|2024-08-29| 3022|
|        9|2025-02-10|2024-08-29| 1969|
|       10|2025-02-10|2024-08-29|  227|
|       11|2025-02-10|2024-08-29| 6387|
|       12|2025-02-10|2024-08-29| 5111|
|       13|2025-02-10|2024-08-29| 2449|
|       14|2025-02-10|2024-08-29| 2675|
|       15|2025-02-10|2024-08-29| 2005|
|       16|2025-02-10|2024-08-29|  784|
|       17|2025-02-10|2024-08-29|  137|
|       18|2025-02-10|2024-08-29|  780|
|       19|2025-02-10|2024-08-29| 2236|
|       20|2025-02-10|2024-08-29| 3630|
+---------+----------+----------+-----+
only showing top 20 rows


In [44]:
second_table_spark_df_transformed.printSchema()

root
 |-- metric_id: integer (nullable = true)
 |-- create_dt: date (nullable = true)
 |-- Date: date (nullable = true)
 |-- Value: integer (nullable = true)


In [140]:
second_output_path = "s3://cci-dig-aicoe-data-sb/processed/healthscore/healthscore_actual_data/"
secondtable_writedf = second_table_spark_df_transformed.coalesce(1).write.format("parquet").mode("append").save(second_output_path)

In [36]:
#second_table_df_melted.to_csv(r'D:\Healthsore Data\Table2.csv', index = False)

In [37]:
#sample_df = df[df['feature_name'].isin(['SMART HELP', 'RESET MODEM', 'RESET TV BOX', 'LOG IN', 'BIL LING', 'OVER ALL', 'EASY CON NECT', 'DATA USAGE', 'RE GISTRATION (NATIVE)'])]
#sample_df.shape

(218, 111)

In [50]:
df.dtypes

metrics                   object
operating_system_type     object
29-08-2024                object
30-08-2024                object
31-08-2024                object
                          ...   
level_no                   int64
feature_id                 int64
feature_seq_num           object
create_dt                 object
parent_id                float64
Length: 111, dtype: object


In [51]:
import pandas as pd

def calculate_percentage(df, datecolumns, metricid, parentid):
    # Ensure these columns exist in the DataFrame
    if metricid not in df.columns or parentid not in df.columns:
        raise ValueError(f"'{metricid}' or '{parentid}' columns not found in DataFrame.")
    
    # Create a copy of the DataFrame to avoid setting on a view
    df = df.copy()
    
    # Iterate through each row and calculate the percentage for each DATE column
    for idx, row in df.iterrows():
        # Iterate through all the DATE columns
        for date_col in datecolumns:
            # Get the numerator (DATE value) and ensure it is a number
            numerator = pd.to_numeric(row[date_col], errors='coerce')
            
            # Get the parent hierarchy ID
            parent_id = row[parentid]
            
            # Check if the parent is empty
            if pd.isna(parent_id) or parent_id == "":
                # If the PARENTID is empty, keep the original DATE value
                percentage = numerator  # Assuming 100% when no parent is found
            else:
                # Find the row where HIERARCHYID == parent_hierarchyid to get the parent's DATE value
                parent_row = df[df[metricid] == parent_id]
                
                # If the parent exists, get the parent's DATE value for the current DATE column
                if not parent_row.empty:
                    denominator = pd.to_numeric(parent_row[date_col].values[0], errors='coerce')
                else:
                    denominator = 0  # In case no parent is found, avoid division by zero
                
                # Calculate the percentage
                if denominator != 0:
                    percentage = (numerator / denominator) * 100
                else:
                    percentage = 0  # To handle division by zero if no parent found
                
            # Round to 3 decimal places (only after confirming it's a valid number)
            if isinstance(percentage, (int, float)):
                percentage = round(percentage, 3)
            
            # Add the percentage to the DataFrame in a new column
            percentage_column_name = f"{date_col} PERCENTAGE"
            if percentage_column_name not in df.columns:
                df[percentage_column_name] = None  # Initialize the column safely
            
            # Set the percentage for the current row and current date column using .loc
            df.loc[idx, percentage_column_name] = percentage

    return df

In [52]:
percentage_df = calculate_percentage(df, date_columns, 'metric_id', 'parent_id')
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)
percentage_df.head()

                            metrics operating_system_type 29-08-2024  \
0         DR_Visits to Billing Home             Apple iOS      71742   
1         DR_Billing Make a Payment             Apple iOS      28280   
2  DR_Client Errors on Make Payment             Apple iOS          1   
3  DR_Server Errors on Make Payment             Apple iOS          1   
4                 DR_Payment Submit             Apple iOS      26212   

  30-08-2024 31-08-2024 01-09-2024 02-09-2024 03-09-2024 04-09-2024  \
0      76236      54132      52596      54447      68414      70355   
1      33162      20184      19342      18744      23074      25597   
2          0          1          1          0          0          0   
3          3          3          2          0          1          3   
4      31110      19019      18204      17326      20983      23423   

  05-09-2024 06-09-2024 07-09-2024 08-09-2024 09-09-2024 10-09-2024  \
0      79002      78610      50904      47500      59885      60599  

In [53]:
percentage_df.shape

(463, 201)


In [54]:
id_columns = ['create_dt', 'metric_id', 'feature_id', 'feature_seq_num', 'metric_sequence_num', 'metric_seqno','level_no', 'parent_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names', 'operating_system_type', 'metric_nature']

In [55]:
# List of columns that contain "PERCENTAGE"
percentage_columns = [col for col in percentage_df.columns if 'PERCENTAGE' in col]

In [56]:
percent_df = percentage_df[id_columns + percentage_columns]

In [57]:
# Rename columns to remove the word "PERCENTAGE"
percent_df.columns = [col.replace(" PERCENTAGE", "") if "PERCENTAGE" in col else col for col in percent_df.columns]

In [58]:
percent_df.shape

(463, 104)


In [59]:
percent_df.head()

    create_dt  metric_id  feature_id feature_seq_num  metric_sequence_num  \
0  2025-02-10          1           1               3                    1   
1  2025-02-10          2           1               3                    2   
2  2025-02-10          3           1               3                    3   
3  2025-02-10          4           1               3                    4   
4  2025-02-10          5           1               3                    5   

  metric_seqno  level_no  parent_id unique_identifier feature_name  \
0            1         1        NaN                BL     BIL LING   
1            2         2        1.0                BL     BIL LING   
2            5         3        2.0                BL     BIL LING   
3            6         3        2.0                BL     BIL LING   
4            3         2        1.0                BL     BIL LING   

                            metrics                  display_names  \
0         DR_Visits to Billing Home         Bi

In [60]:
def calculate_both(df, metricid, parentid):
    """
    Generate new rows for each unique display_name based on given conditions.

    Args:
    - df (DataFrame): Input DataFrame containing the original data.

    Returns:
    - DataFrame: Updated DataFrame with newly calculated rows.
    """
    new_rows = []  # Store new rows
    seq_num = 1  # Sequential ID for metric_id

    for display_name in df['display_names'].unique():
        # Filter rows for the current display_name
        display_df = df[df['display_names'] == display_name]
            
        # Case 1: If `parent_id_old` has null or empty values
        if display_df[parentid].isnull().any() or display_df[parentid].eq('').any():
            # Sum of `Apple iOS` and `Google Android` for `parent_id_old` being null
            ios_rows = display_df[display_df["operating_system_type"] == "Apple iOS"]
            android_rows = display_df[display_df["operating_system_type"] == "Google Android"]
            
            # Calculate numerator using `where` to avoid downcasting issues
            ios_sums = ios_rows[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            android_sums = android_rows[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            numerator = ios_sums + android_sums
            
            # Store numerator in new row, rounded to 3 decimal places
            new_row_values = numerator.sum(axis=0).fillna(0).apply(lambda x: round(x, 3)).to_dict()
        else:
            # Case 2: If `parent_id_old` is not null
            ios_rows = display_df[(display_df["operating_system_type"] == "Apple iOS") & 
                                  (display_df[parentid].notna()) & 
                                  (display_df[parentid] != '')]
            android_rows = display_df[(display_df["operating_system_type"] == "Google Android") & 
                                      (display_df[parentid].notna()) & 
                                      (display_df[parentid] != '')]
            
            # Calculate numerator using `where` to avoid downcasting issues
            ios_sums = ios_rows[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            android_sums = android_rows[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            numerator = ios_sums + android_sums
            
            # Filter rows where `parent_id_old` matches `hierarchy_id`
            filtered_data = df[df[metricid].isin(display_df[parentid])]
            ios_data = filtered_data[filtered_data['operating_system_type'] == "Apple iOS"]
            android_data = filtered_data[filtered_data['operating_system_type'] == "Google Android"]

            # Calculate denominator
            ios_sums_denom = ios_data[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            android_sums_denom = android_data[date_columns].apply(pd.to_numeric, errors='coerce').reset_index(drop=True)
            denominator = ios_sums_denom + android_sums_denom

            # Calculate result as percentage
            result = (numerator / denominator) * 100 if not denominator.empty else pd.DataFrame(0, columns=date_columns, index=numerator.index)
            result = result.round(3)  # Round result to 3 decimal places
            new_row_values = result.sum(axis=0).fillna(0).apply(lambda x: round(x, 3)).to_dict()

        # Create the new row for the current display_name
        new_row = {
            'display_names': display_name,
            'operating_system_type': 'Both',
            'metric_id': f'100000{seq_num}'
        }
        # Add calculated values to the new row
        new_row.update(new_row_values)
        new_rows.append(new_row)
        seq_num += 1

    # Append new rows to the original DataFrame
    new_rows_df = pd.DataFrame(new_rows)
    result_df = pd.concat([df, new_rows_df], ignore_index=True)

    # Reset index
    result_df.reset_index(drop=True, inplace=True)
    return new_rows_df


In [61]:
both_calc_df = calculate_both(df,'metric_id', 'parent_id')
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 300)      # Show 100 rows
both_calc_df.head(5)

                   display_names operating_system_type metric_id  29-08-2024  \
0         Billing Section Visits                  Both   1000001   93064.000   
1                  Make Payments                  Both   1000002      39.606   
2  Client Errors on Payment Page                  Both   1000003       0.003   
3  Server Errors on Payment Page                  Both   1000004       0.263   
4                 Payment Submit                  Both   1000005      36.462   

   30-08-2024  31-08-2024  01-09-2024  02-09-2024  03-09-2024  04-09-2024  \
0   99690.000   70846.000   68476.000   70777.000   89457.000   91825.000   
1      44.064      37.685      36.988      34.547      34.328      36.616   
2       0.000       0.004       0.004       0.000       0.000       0.000   
3       0.355       0.360       0.276       0.344       0.374       0.404   
4      41.049      34.924      34.270      31.465      30.872      33.069   

   05-09-2024  06-09-2024  07-09-2024  08-09-2024  09-09

In [62]:
both_calc_df.shape

(251, 93)


In [63]:
unique_display_names_df_test = df.drop_duplicates(subset='display_names')
unique_display_names_df_test.shape

(251, 111)


In [64]:
id_columns_both_test = ['create_dt', 'feature_id', 'feature_seq_num', 'metric_sequence_num', 'metric_seqno', 'parent_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names','metric_nature' ]
id_column_both_df_test = unique_display_names_df_test[id_columns_both_test]
#id_column_both_df = id_column_both_df[id_column_both_df['feature_name'].isin(['SMART HELP'])]
id_column_both_df_test.shape

(251, 11)


In [65]:
# Perform a left merge on 'display_names'
both_df_test =  pd.merge(both_calc_df, id_column_both_df_test, on='display_names', how='inner')
both_df_test.shape

(251, 103)


In [66]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 300)      # Show 100 rows
both_df_test.head()

                   display_names operating_system_type metric_id  29-08-2024  \
0         Billing Section Visits                  Both   1000001   93064.000   
1                  Make Payments                  Both   1000002      39.606   
2  Client Errors on Payment Page                  Both   1000003       0.003   
3  Server Errors on Payment Page                  Both   1000004       0.263   
4                 Payment Submit                  Both   1000005      36.462   

   30-08-2024  31-08-2024  01-09-2024  02-09-2024  03-09-2024  04-09-2024  \
0   99690.000   70846.000   68476.000   70777.000   89457.000   91825.000   
1      44.064      37.685      36.988      34.547      34.328      36.616   
2       0.000       0.004       0.004       0.000       0.000       0.000   
3       0.355       0.360       0.276       0.344       0.374       0.404   
4      41.049      34.924      34.270      31.465      30.872      33.069   

   05-09-2024  06-09-2024  07-09-2024  08-09-2024  09-09

In [67]:
print(list(both_df_test))

['display_names', 'operating_system_type', 'metric_id', '29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024', '23-10-2024', '24-10-2024', '25-10-2024', '26-10-2024', '27-10-2024', '28-10-2024', '29-10-2024', '30-10-2024', '31-10-2024', '01-11-2024', '02-11-2024', '03-11-2024', '04-11

In [68]:
both_df_test.shape

(251, 103)


In [69]:
modifiled_first_table_df = pd.concat([df, both_df_test], ignore_index=True)

In [70]:
modifiled_first_table_df.shape

(714, 111)


In [71]:
print(list(modifiled_first_table_df))

['metrics', 'operating_system_type', '29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024', '23-10-2024', '24-10-2024', '25-10-2024', '26-10-2024', '27-10-2024', '28-10-2024', '29-10-2024', '30-10-2024', '31-10-2024', '01-11-2024', '02-11-2024', '03-11-2024', '04-11-2024', '05-11-2024

In [72]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 500)      # Show 100 rows
modifiled_first_table_df = modifiled_first_table_df[['metric_id', 'metrics', 'display_names', 'operating_system_type', 'feature_name', 'unique_identifier', 'metric_nature', 'level_1', 'level_2', 'level_3', 'level_4', 'level_5', 'hierarchy_id', 'parent_id_old', 'metric_sequence_num', 'metric_seqno', 'level_no', 'feature_id', 'feature_seq_num', 'parent_id', 'create_dt']]
modifiled_first_table_df.head()

  metric_id                           metrics                  display_names  \
0         1         DR_Visits to Billing Home         Billing Section Visits   
1         2         DR_Billing Make a Payment                  Make Payments   
2         3  DR_Client Errors on Make Payment  Client Errors on Payment Page   
3         4  DR_Server Errors on Make Payment  Server Errors on Payment Page   
4         5                 DR_Payment Submit                 Payment Submit   

  operating_system_type feature_name unique_identifier metric_nature  \
0             Apple iOS     BIL LING                BL      Positive   
1             Apple iOS     BIL LING                BL      Positive   
2             Apple iOS     BIL LING                BL      Negative   
3             Apple iOS     BIL LING                BL      Negative   
4             Apple iOS     BIL LING                BL      Positive   

                  level_1         level_2                        level_3  \
0  Billing

In [73]:
first_table_spark_df = spark.createDataFrame(modifiled_first_table_df)

In [74]:
first_table_spark_df.printSchema()

root
 |-- metric_id: string (nullable = true)
 |-- metrics: string (nullable = true)
 |-- display_names: string (nullable = true)
 |-- operating_system_type: string (nullable = true)
 |-- feature_name: string (nullable = true)
 |-- unique_identifier: string (nullable = true)
 |-- metric_nature: string (nullable = true)
 |-- level_1: string (nullable = true)
 |-- level_2: string (nullable = true)
 |-- level_3: string (nullable = true)
 |-- level_4: string (nullable = true)
 |-- level_5: string (nullable = true)
 |-- hierarchy_id: string (nullable = true)
 |-- parent_id_old: string (nullable = true)
 |-- metric_sequence_num: long (nullable = true)
 |-- metric_seqno: string (nullable = true)
 |-- level_no: double (nullable = true)
 |-- feature_id: long (nullable = true)
 |-- feature_seq_num: string (nullable = true)
 |-- parent_id: double (nullable = true)
 |-- create_dt: date (nullable = true)


In [75]:
from pyspark.sql import functions as F

# Convert the specified columns to integer type
first_table_spark_df_transformed = first_table_spark_df \
    .withColumn("metric_id", F.col("metric_id").cast("int")) \
    .withColumn("metric_sequence_num", F.col("metric_sequence_num").cast("int")) \
    .withColumn("metric_seqno", F.col("metric_seqno").cast("int")) \
    .withColumn("level_no", F.col("level_no").cast("int")) \
    .withColumn("feature_id", F.col("feature_id").cast("int")) \
    .withColumn("feature_seq_num", F.col("feature_seq_num").cast("int")) \
    .withColumn("parent_id", F.col("parent_id").cast("int"))

# Show the updated DataFrame to verify the changes
first_table_spark_df_transformed.show()


+---------+--------------------+--------------------+---------------------+------------+-----------------+-------------+--------------------+--------------------+--------------------+-------+-------+------------------+----------------+-------------------+------------+--------+----------+---------------+---------+----------+
|metric_id|             metrics|       display_names|operating_system_type|feature_name|unique_identifier|metric_nature|             level_1|             level_2|             level_3|level_4|level_5|      hierarchy_id|   parent_id_old|metric_sequence_num|metric_seqno|level_no|feature_id|feature_seq_num|parent_id| create_dt|
+---------+--------------------+--------------------+---------------------+------------+-----------------+-------------+--------------------+--------------------+--------------------+-------+-------+------------------+----------------+-------------------+------------+--------+----------+---------------+---------+----------+
|        1|DR_Visits t

In [76]:
first_table_spark_df_transformed.printSchema()

root
 |-- metric_id: integer (nullable = true)
 |-- metrics: string (nullable = true)
 |-- display_names: string (nullable = true)
 |-- operating_system_type: string (nullable = true)
 |-- feature_name: string (nullable = true)
 |-- unique_identifier: string (nullable = true)
 |-- metric_nature: string (nullable = true)
 |-- level_1: string (nullable = true)
 |-- level_2: string (nullable = true)
 |-- level_3: string (nullable = true)
 |-- level_4: string (nullable = true)
 |-- level_5: string (nullable = true)
 |-- hierarchy_id: string (nullable = true)
 |-- parent_id_old: string (nullable = true)
 |-- metric_sequence_num: integer (nullable = true)
 |-- metric_seqno: integer (nullable = true)
 |-- level_no: integer (nullable = true)
 |-- feature_id: integer (nullable = true)
 |-- feature_seq_num: integer (nullable = true)
 |-- parent_id: integer (nullable = true)
 |-- create_dt: date (nullable = true)


In [139]:
first_output_path = "s3://cci-dig-aicoe-data-sb/processed/healthscore/healthscore_master/"
first_write_df = first_table_spark_df_transformed.coalesce(1).write.format("parquet").mode("append").save(first_output_path)

In [59]:
#modifiled_first_table_df.to_csv(r'D:\Healthsore Data\Table1.csv', index = False)

In [78]:
unique_display_names_df = df.drop_duplicates(subset='display_names')
unique_display_names_df.shape

(251, 111)


In [79]:
id_columns_both = ['create_dt', 'feature_id', 'feature_seq_num', 'metric_sequence_num', 'metric_seqno', 'parent_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names','metric_nature' ]
id_column_both_df = unique_display_names_df[id_columns_both]
#id_column_both_df = id_column_both_df[id_column_both_df['feature_name'].isin(['SMART HELP'])]
id_column_both_df.shape

(251, 11)


In [80]:
# Perform a left merge on 'display_names'
both_df =  pd.merge(both_calc_df, id_column_both_df, on='display_names', how='inner')
both_df.shape

(251, 103)


In [81]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 300)      # Show 100 rows
both_df.head()


                   display_names operating_system_type metric_id  29-08-2024  \
0         Billing Section Visits                  Both   1000001   93064.000   
1                  Make Payments                  Both   1000002      39.606   
2  Client Errors on Payment Page                  Both   1000003       0.003   
3  Server Errors on Payment Page                  Both   1000004       0.263   
4                 Payment Submit                  Both   1000005      36.462   

   30-08-2024  31-08-2024  01-09-2024  02-09-2024  03-09-2024  04-09-2024  \
0   99690.000   70846.000   68476.000   70777.000   89457.000   91825.000   
1      44.064      37.685      36.988      34.547      34.328      36.616   
2       0.000       0.004       0.004       0.000       0.000       0.000   
3       0.355       0.360       0.276       0.344       0.374       0.404   
4      41.049      34.924      34.270      31.465      30.872      33.069   

   05-09-2024  06-09-2024  07-09-2024  08-09-2024  09-09

In [82]:
third_table_df = pd.concat([percent_df, both_df], ignore_index=True)

In [83]:
third_table_df.shape

(714, 104)


In [84]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 200)      # Show 100 rows
third_table_df.head()

    create_dt metric_id  feature_id feature_seq_num  metric_sequence_num  \
0  2025-02-10         1           1               3                    1   
1  2025-02-10         2           1               3                    2   
2  2025-02-10         3           1               3                    3   
3  2025-02-10         4           1               3                    4   
4  2025-02-10         5           1               3                    5   

  metric_seqno  level_no  parent_id unique_identifier feature_name  \
0            1       1.0        NaN                BL     BIL LING   
1            2       2.0        1.0                BL     BIL LING   
2            5       3.0        2.0                BL     BIL LING   
3            6       3.0        2.0                BL     BIL LING   
4            3       2.0        1.0                BL     BIL LING   

                            metrics                  display_names  \
0         DR_Visits to Billing Home         Billing 

In [85]:
print(list(third_table_df))

['create_dt', 'metric_id', 'feature_id', 'feature_seq_num', 'metric_sequence_num', 'metric_seqno', 'level_no', 'parent_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names', 'operating_system_type', 'metric_nature', '29-08-2024', '30-08-2024', '31-08-2024', '01-09-2024', '02-09-2024', '03-09-2024', '04-09-2024', '05-09-2024', '06-09-2024', '07-09-2024', '08-09-2024', '09-09-2024', '10-09-2024', '11-09-2024', '12-09-2024', '13-09-2024', '14-09-2024', '15-09-2024', '16-09-2024', '17-09-2024', '18-09-2024', '19-09-2024', '20-09-2024', '21-09-2024', '22-09-2024', '23-09-2024', '24-09-2024', '25-09-2024', '26-09-2024', '27-09-2024', '28-09-2024', '29-09-2024', '30-09-2024', '01-10-2024', '02-10-2024', '03-10-2024', '04-10-2024', '05-10-2024', '06-10-2024', '07-10-2024', '08-10-2024', '09-10-2024', '10-10-2024', '11-10-2024', '12-10-2024', '13-10-2024', '14-10-2024', '15-10-2024', '16-10-2024', '17-10-2024', '18-10-2024', '19-10-2024', '20-10-2024', '21-10-2024', '22-10-2024',

In [86]:
id_columns = ['create_dt', 'metric_id', 'feature_id', 'feature_seq_num', 'metric_sequence_num', 'metric_seqno', 'level_no', 'parent_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names', 'operating_system_type', 'metric_nature']

In [10]:
#third_table_df.to_csv(r'D:\Healthsore Data\Table3.csv', index = False)

In [87]:
third_table_melted_df = pd.melt(third_table_df, id_vars=id_columns, 
                     value_vars=date_columns, 
                     var_name='Date', 
                     value_name='Value')

In [88]:
third_table_melted_df.shape

(64260, 16)


In [89]:
third_table_melted_df.columns

Index(['create_dt', 'metric_id', 'feature_id', 'feature_seq_num',
       'metric_sequence_num', 'metric_seqno', 'level_no', 'parent_id',
       'unique_identifier', 'feature_name', 'metrics', 'display_names',
       'operating_system_type', 'metric_nature', 'Date', 'Value'],
      dtype='object')


In [90]:
third_table_final_df = third_table_melted_df[['create_dt', 'metric_id', 'feature_id', 'feature_name', 'metrics', 'display_names', 'operating_system_type', 'Date', 'Value']]

In [91]:
third_table_final_df.head()

    create_dt metric_id  feature_id feature_name  \
0  2025-02-10         1           1     BIL LING   
1  2025-02-10         2           1     BIL LING   
2  2025-02-10         3           1     BIL LING   
3  2025-02-10         4           1     BIL LING   
4  2025-02-10         5           1     BIL LING   

                            metrics                  display_names  \
0         DR_Visits to Billing Home         Billing Section Visits   
1         DR_Billing Make a Payment                  Make Payments   
2  DR_Client Errors on Make Payment  Client Errors on Payment Page   
3  DR_Server Errors on Make Payment  Server Errors on Payment Page   
4                 DR_Payment Submit                 Payment Submit   

  operating_system_type        Date   Value  
0             Apple iOS  29-08-2024   71742  
1             Apple iOS  29-08-2024  39.419  
2             Apple iOS  29-08-2024   0.004  
3             Apple iOS  29-08-2024   0.004  
4             Apple iOS  29-08-2024 

In [92]:
third_table_final_df.shape

(64260, 9)


In [97]:
third_table_final_df.dtypes

create_dt                object
metric_id                object
feature_id                int64
feature_name             object
metrics                  object
display_names            object
operating_system_type    object
Date                     object
Value                    object
dtype: object


In [98]:
import pandas as pd

# Convert 'Value' to numeric type in Pandas (handle errors if any non-numeric values are present)
third_table_final_df['Value'] = pd.to_numeric(third_table_final_df['Value'], errors='coerce')

# Convert 'Date' to datetime format in Pandas
third_table_final_df['Date'] = pd.to_datetime(third_table_final_df['Date'], format='%d-%m-%Y')

# Convert 'create_dt' to datetime format in Pandas (if needed)
third_table_final_df['create_dt'] = pd.to_datetime(third_table_final_df['create_dt'])

# Check the DataFrame to make sure the conversions are done
third_table_final_df.dtypes


create_dt                datetime64[ns]
metric_id                        object
feature_id                        int64
feature_name                     object
metrics                          object
display_names                    object
operating_system_type            object
Date                     datetime64[ns]
Value                           float64
dtype: object
<stdin>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<stdin>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<stdin>:10: SettingWithCopyW

In [100]:
third_table_final_df.head()

   create_dt metric_id  feature_id feature_name  \
0 2025-02-10         1           1     BIL LING   
1 2025-02-10         2           1     BIL LING   
2 2025-02-10         3           1     BIL LING   
3 2025-02-10         4           1     BIL LING   
4 2025-02-10         5           1     BIL LING   

                            metrics                  display_names  \
0         DR_Visits to Billing Home         Billing Section Visits   
1         DR_Billing Make a Payment                  Make Payments   
2  DR_Client Errors on Make Payment  Client Errors on Payment Page   
3  DR_Server Errors on Make Payment  Server Errors on Payment Page   
4                 DR_Payment Submit                 Payment Submit   

  operating_system_type       Date      Value  
0             Apple iOS 2024-08-29  71742.000  
1             Apple iOS 2024-08-29     39.419  
2             Apple iOS 2024-08-29      0.004  
3             Apple iOS 2024-08-29      0.004  
4             Apple iOS 2024-08-

In [101]:
third_table_spark_df = spark.createDataFrame(third_table_final_df)

In [102]:
third_table_spark_df.show()

+-------------------+---------+----------+------------+--------------------+--------------------+---------------------+-------------------+-------+
|          create_dt|metric_id|feature_id|feature_name|             metrics|       display_names|operating_system_type|               Date|  Value|
+-------------------+---------+----------+------------+--------------------+--------------------+---------------------+-------------------+-------+
|2025-02-10 00:00:00|        1|         1|    BIL LING|DR_Visits to Bill...|Billing Section V...|            Apple iOS|2024-08-29 00:00:00|71742.0|
|2025-02-10 00:00:00|        2|         1|    BIL LING|DR_Billing Make a...|       Make Payments|            Apple iOS|2024-08-29 00:00:00| 39.419|
|2025-02-10 00:00:00|        3|         1|    BIL LING|DR_Client Errors ...|Client Errors on ...|            Apple iOS|2024-08-29 00:00:00|  0.004|
|2025-02-10 00:00:00|        4|         1|    BIL LING|DR_Server Errors ...|Server Errors on ...|            App

In [103]:
third_table_spark_df.printSchema()

root
 |-- create_dt: timestamp (nullable = true)
 |-- metric_id: string (nullable = true)
 |-- feature_id: long (nullable = true)
 |-- feature_name: string (nullable = true)
 |-- metrics: string (nullable = true)
 |-- display_names: string (nullable = true)
 |-- operating_system_type: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Value: double (nullable = true)


In [104]:
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, FloatType, DateType

# Convert columns to the desired types
third_table_spark_df_transformed = third_table_spark_df \
    .withColumn("create_dt", F.to_date(F.col("create_dt"))) \
    .withColumn("metric_id", F.col("metric_id").cast(IntegerType())) \
    .withColumn("feature_id", F.col("feature_id").cast(IntegerType())) \
    .withColumn("Date", F.to_date(F.col("Date"))) \
    .withColumn("Value", F.col("Value").cast(FloatType()))

# Show the updated DataFrame to verify the changes
third_table_spark_df_transformed.show()



+----------+---------+----------+------------+--------------------+--------------------+---------------------+----------+-------+
| create_dt|metric_id|feature_id|feature_name|             metrics|       display_names|operating_system_type|      Date|  Value|
+----------+---------+----------+------------+--------------------+--------------------+---------------------+----------+-------+
|2025-02-10|        1|         1|    BIL LING|DR_Visits to Bill...|Billing Section V...|            Apple iOS|2024-08-29|71742.0|
|2025-02-10|        2|         1|    BIL LING|DR_Billing Make a...|       Make Payments|            Apple iOS|2024-08-29| 39.419|
|2025-02-10|        3|         1|    BIL LING|DR_Client Errors ...|Client Errors on ...|            Apple iOS|2024-08-29|  0.004|
|2025-02-10|        4|         1|    BIL LING|DR_Server Errors ...|Server Errors on ...|            Apple iOS|2024-08-29|  0.004|
|2025-02-10|        5|         1|    BIL LING|   DR_Payment Submit|      Payment Submit|  

In [105]:
# Verify the schema
third_table_spark_df_transformed.printSchema()

root
 |-- create_dt: date (nullable = true)
 |-- metric_id: integer (nullable = true)
 |-- feature_id: integer (nullable = true)
 |-- feature_name: string (nullable = true)
 |-- metrics: string (nullable = true)
 |-- display_names: string (nullable = true)
 |-- operating_system_type: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- Value: float (nullable = true)


In [106]:
third_output_path = "s3://cci-dig-aicoe-data-sb/processed/healthscore/healthscore_percent_both/"
third_table_spark_df_transformed.coalesce(1).write.format("parquet").mode("append").save(third_output_path)

In [76]:
#third_table_final_df.to_csv(r'D:\Healthsore Data\Table3_athena.csv', index = False)

In [107]:
third_table_melted_df.dtypes

create_dt                 object
metric_id                 object
feature_id                 int64
feature_seq_num           object
metric_sequence_num        int64
metric_seqno              object
level_no                 float64
parent_id                float64
unique_identifier         object
feature_name              object
metrics                   object
display_names             object
operating_system_type     object
metric_nature             object
Date                      object
Value                     object
dtype: object


In [108]:
# Convert 'create_dt' and 'Date' to datetime
third_table_melted_df['create_dt'] = pd.to_datetime(third_table_melted_df['create_dt'], errors='coerce')
third_table_melted_df['Date'] = pd.to_datetime(third_table_melted_df['Date'], errors='coerce')

# Convert 'feature_id', 'metric_sequence_num', 'level_no', 'parent_id' to integers
third_table_melted_df['feature_id'] = third_table_melted_df['feature_id'].astype('Int64')  # 'Int64' to handle missing values (NaN)
third_table_melted_df['feature_seq_num'] = third_table_melted_df['feature_seq_num'].astype('Int64')
third_table_melted_df['metric_sequence_num'] = third_table_melted_df['metric_sequence_num'].astype('Int64')
#third_table_melted_df['metric_seqno'] = third_table_melted_df['metric_seqno'].astype('Int64')
third_table_melted_df['level_no'] = third_table_melted_df['level_no'].astype('Int64')
third_table_melted_df['parent_id'] = third_table_melted_df['parent_id'].astype('Int64')
#third_table_melted_df['metric_id'] = third_table_melted_df['metric_id'].astype('Int64')

# Convert 'metric_id', 'unique_identifier', 'feature_name', 'metrics', 'display_names', 'operating_system_type' to strings
third_table_melted_df['metric_id'] = third_table_melted_df['metric_id'].astype(str)
third_table_melted_df['unique_identifier'] = third_table_melted_df['unique_identifier'].astype(str)
third_table_melted_df['feature_name'] = third_table_melted_df['feature_name'].astype(str)
third_table_melted_df['metrics'] = third_table_melted_df['metrics'].astype(str)
third_table_melted_df['display_names'] = third_table_melted_df['display_names'].astype(str)
third_table_melted_df['operating_system_type'] = third_table_melted_df['operating_system_type'].astype(str)
third_table_melted_df['metric_nature'] = third_table_melted_df['metric_nature'].astype(str)

# Convert 'Value' to float
third_table_melted_df['Value'] = pd.to_numeric(third_table_melted_df['Value'], errors='coerce')

<stdin>:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.


In [109]:
third_table_melted_df.dtypes

create_dt                datetime64[ns]
metric_id                        object
feature_id                        Int64
feature_seq_num                   Int64
metric_sequence_num               Int64
metric_seqno                     object
level_no                          Int64
parent_id                         Int64
unique_identifier                object
feature_name                     object
metrics                          object
display_names                    object
operating_system_type            object
metric_nature                    object
Date                     datetime64[ns]
Value                           float64
dtype: object


In [110]:
#Calculate 1st and 3rd quartiles
quartiles = third_table_melted_df.groupby(['operating_system_type', 'display_names'])['Value'].quantile([0.25, 0.75]).unstack()
quartiles.reset_index(inplace=True)
quartiles.columns.name = None  # Remove the index name
quartiles.columns = ['operating_system_type', 'display_names', '1st Quartile', '3rd Quartile']
#Calculate IQR
quartiles['IQR'] = quartiles['3rd Quartile'] - quartiles['1st Quartile']

In [111]:
quartiles.shape

(708, 5)


In [112]:
quartiles_iqr_df = pd.merge(third_table_df, quartiles, on=['display_names', 'operating_system_type'], how='outer')

In [113]:
quartiles_iqr_df.shape

(714, 107)


In [114]:
pd.set_option('display.max_columns', None)  # Show all columns 
pd.set_option('display.max_rows', 200)      # Show 100 rows 
quartiles_iqr_df.head()

    create_dt metric_id  feature_id feature_seq_num  metric_sequence_num  \
0  2025-02-10        22           2              13                   22   
1  2025-02-10        23           2              13                   23   
2  2025-02-10        74           5               4                   74   
3  2025-02-10        75           5               4                   75   
4  2025-02-10  10000022           2              13                   22   

  metric_seqno  level_no  parent_id unique_identifier   feature_name  \
0                    1.0        NaN                CM     COX MOBILE   
1                    1.0        NaN                CM     COX MOBILE   
2                    1.0        NaN               ECN  EASY CON NECT   
3                    1.0        NaN               ECN  EASY CON NECT   
4                    NaN        NaN                CM     COX MOBILE   

                                  metrics display_names operating_system_type  \
0                  DR_Mobile 

In [115]:

def calculate_upper_lower(df, parent_column='parent_id'):
    # Ensure parent_column is present in the dataframe
    if parent_column not in df.columns:
        raise ValueError(f"{parent_column} not found in the dataframe")
    
    # Define the function to apply to each row
    def calculate_row(row):
        if pd.isna(row[parent_column]) or row[parent_column] == '':
            # Apply 1.25 logic for rows where parent_id is null, empty, or None
            upper = row['3rd Quartile'] + (1.25 * row['IQR'])
            lower = row['1st Quartile'] - (1.25 * row['IQR'])
        else:
            # Apply 0.75 logic for other rows
            upper = row['3rd Quartile'] + (0.75 * row['IQR'])
            lower = row['1st Quartile'] - (0.75 * row['IQR'])
        
        return pd.Series({'Upper': upper, 'Lower': lower})
    
    # Apply the function row by row
    df[['Upper', 'Lower']] = df.apply(calculate_row, axis=1)

    # Round the Upper column to 1 decimal place
    #df['Upper'] = df['Upper'].round(1)

    return df


In [116]:
upper_lower_df = calculate_upper_lower(quartiles_iqr_df)
#upper_lower_df =upper_lower_df[['display_names','operating_system_type','1st Quartile','3rd Quartile', 'IQR', 'Upper', 'Lower']]
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 200)      # Show 100 rows
upper_lower_df.head()

    create_dt metric_id  feature_id feature_seq_num  metric_sequence_num  \
0  2025-02-10        22           2              13                   22   
1  2025-02-10        23           2              13                   23   
2  2025-02-10        74           5               4                   74   
3  2025-02-10        75           5               4                   75   
4  2025-02-10  10000022           2              13                   22   

  metric_seqno  level_no  parent_id unique_identifier   feature_name  \
0                    1.0        NaN                CM     COX MOBILE   
1                    1.0        NaN                CM     COX MOBILE   
2                    1.0        NaN               ECN  EASY CON NECT   
3                    1.0        NaN               ECN  EASY CON NECT   
4                    NaN        NaN                CM     COX MOBILE   

                                  metrics display_names operating_system_type  \
0                  DR_Mobile 

In [117]:
upper_lower_df.shape

(714, 109)


In [118]:
import numpy as np
import pandas as pd

def calculate_last7_and_30_days(df):
    # Extract the date columns (skip the non-date columns like 'display_names' and 'operating_system_type')
    datecolumn = [col for col in df.columns if '-' in col]
    
    # Exclude the latest date column (the first one)
    last_7_columns = datecolumn[-8:-1]  # Get the last 7 columns excluding the latest date
    last_30_columns = datecolumn[-31:-1]

    # Extract the date part and convert them to datetime objects
    date_objects = [pd.to_datetime(col.split()[0], format='%d-%m-%Y') for col in datecolumn]

    # Get the column corresponding to the latest date
    latest_date = max(date_objects)
    latest_date_column = datecolumn[date_objects.index(latest_date)]

    # Create a new column 'Yesterday' with the values from the latest date column
    df.loc[:, 'Yesterday'] = df[latest_date_column]
    
    # Ensure numeric columns before performing mean calculation
    df[last_7_columns] = df[last_7_columns].apply(pd.to_numeric, errors='coerce')
    df[last_30_columns] = df[last_30_columns].apply(pd.to_numeric, errors='coerce')

    # Calculate the mean for each row across the last 7 and 30 date columns
    df.loc[:, 'last_7_days'] = df[last_7_columns].mean(axis=1)
    df.loc[:, 'last_30_days'] = df[last_30_columns].mean(axis=1)

    # Replace 0 and NaN values in 'last_7_days' and 'last_30_days' with NaN to avoid division by zero
    df['last_7_days'] = df['last_7_days'].replace(0, np.nan)
    df['last_30_days'] = df['last_30_days'].replace(0, np.nan)

    # Calculate the percentage change for 'Last 7 Days' with a check for NaN
    df.loc[:, '% Change Last 7 Days'] = np.where(
        df['last_7_days'].isna(), 0, 
        (df['Yesterday'] - df['last_7_days']) / df['last_7_days'] * 100
    )

    # Calculate the percentage change for 'Last 30 Days' with a check for NaN
    df.loc[:, '% Change Last 30 Days'] = np.where(
        df['last_30_days'].isna(), 0, 
        (df['Yesterday'] - df['last_30_days']) / df['last_30_days'] * 100
    )

    # Round the percentage changes to 1 decimal place
    df.loc[:, '% Change Last 7 Days'] = df['% Change Last 7 Days'].round(1)
    df.loc[:, '% Change Last 30 Days'] = df['% Change Last 30 Days'].round(1)
    
    return df


In [119]:
last_7_and_30_days_df  = calculate_last7_and_30_days(upper_lower_df)
#last_7_and_30_days_df = last_7_and_30_days_df[['metric_id', 'metrics', 'display_names', 'operating_system_type', 'feature_name', 'parent_id','Yesterday', 'last_7_days', 'last_30_days', '% Change Last 7 Days', '% Change Last 30 Days']]
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 100)
last_7_and_30_days_df.head()

    create_dt metric_id  feature_id feature_seq_num  metric_sequence_num  \
0  2025-02-10        22           2              13                   22   
1  2025-02-10        23           2              13                   23   
2  2025-02-10        74           5               4                   74   
3  2025-02-10        75           5               4                   75   
4  2025-02-10  10000022           2              13                   22   

  metric_seqno  level_no  parent_id unique_identifier   feature_name  \
0                    1.0        NaN                CM     COX MOBILE   
1                    1.0        NaN                CM     COX MOBILE   
2                    1.0        NaN               ECN  EASY CON NECT   
3                    1.0        NaN               ECN  EASY CON NECT   
4                    NaN        NaN                CM     COX MOBILE   

                                  metrics display_names operating_system_type  \
0                  DR_Mobile 

In [120]:
last_7_and_30_days_df.shape

(714, 114)


In [121]:
last_7_and_30_days_df[['1st Quartile', '3rd Quartile', 'IQR','Upper', 'Lower', 'Yesterday', 'last_7_days', 'last_30_days', '% Change Last 7 Days', '% Change Last 30 Days']] = last_7_and_30_days_df[['1st Quartile', '3rd Quartile', 'IQR','Upper', 'Lower', 'Yesterday', 'last_7_days', 'last_30_days', '% Change Last 7 Days', '% Change Last 30 Days']].round(3)

In [122]:
last_7_and_30_days_df.head()

    create_dt metric_id  feature_id feature_seq_num  metric_sequence_num  \
0  2025-02-10        22           2              13                   22   
1  2025-02-10        23           2              13                   23   
2  2025-02-10        74           5               4                   74   
3  2025-02-10        75           5               4                   75   
4  2025-02-10  10000022           2              13                   22   

  metric_seqno  level_no  parent_id unique_identifier   feature_name  \
0                    1.0        NaN                CM     COX MOBILE   
1                    1.0        NaN                CM     COX MOBILE   
2                    1.0        NaN               ECN  EASY CON NECT   
3                    1.0        NaN               ECN  EASY CON NECT   
4                    NaN        NaN                CM     COX MOBILE   

                                  metrics display_names operating_system_type  \
0                  DR_Mobile 

In [123]:
last_7_and_30_days_df.shape

(714, 114)


In [124]:
import pandas as pd

def add_colour_indicators(df):
    # Function to assign colour_indicator based on the logic
    def assign_colour_indicator(row):
        if row['metric_nature'] == 'Positive':
            if row['Yesterday'] > row['Upper']:
                return 'Positive Above'
            elif row['Yesterday'] < row['Lower']:
                return 'Positive Below'
            else:
                return 'Positive Within'
        elif row['metric_nature'] == 'Negative':
            if row['Yesterday'] > row['Upper']:
                return 'Negative Above'
            elif row['Yesterday'] < row['Lower']:
                return 'Negative Below'
            else:
                return 'Negative Within'
        return None  # In case metric_nature is neither positive nor negative

    # Function to assign colour_indicator1 based on colour_indicator values
    def assign_colour_indicator1(row):
        if row['metric_nature_indicator'] in ['Positive Above', 'Negative Below']:
            return 'Green'
        elif row['metric_nature_indicator'] in ['Positive Below', 'Negative Above']:
            return 'Red'
        elif row['metric_nature_indicator'] in ['Positive Within', 'Negative Within']:
            return 'White'
        return None  # In case colour_indicator is not set
    
    # Apply the function to create the colour_indicator column
    df['metric_nature_indicator'] = df.apply(assign_colour_indicator, axis=1)
    
    # Apply the function to create the colour_indicator1 column
    df['colour_indicator'] = df.apply(assign_colour_indicator1, axis=1)
    
    return df  # Return the modified DataFrame



In [125]:
# Call the function and get the updated DataFrame
colour_indicators_df  = add_colour_indicators(last_7_and_30_days_df)
colour_indicators_df = colour_indicators_df[['create_dt', 'metric_id', 'feature_name', 'metrics', 'display_names', 'operating_system_type', '1st Quartile', '3rd Quartile', 'IQR', 'Upper', 'Lower', 'Yesterday', 'last_7_days', 'last_30_days', '% Change Last 7 Days', '% Change Last 30 Days', 'metric_nature_indicator', 'colour_indicator']]

# Display the resulting dataframe
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', 500)
colour_indicators_df.head()


    create_dt metric_id   feature_name  \
0  2025-02-10        22     COX MOBILE   
1  2025-02-10        23     COX MOBILE   
2  2025-02-10        74  EASY CON NECT   
3  2025-02-10        75  EASY CON NECT   
4  2025-02-10  10000022     COX MOBILE   

                                  metrics display_names operating_system_type  \
0                  DR_Mobile Lock Enabled                           Apple iOS   
1                 DR_Mobile Lock Disabled                           Apple iOS   
2  DR_Easy Connect - Self install success                           Apple iOS   
3  DR_Easy Connect - Self install Timeout                           Apple iOS   
4                  DR_Mobile Lock Enabled                                Both   

   1st Quartile  3rd Quartile     IQR     Upper    Lower Yesterday  \
0           0.0        143.00  143.00   321.750 -178.750        42   
1           0.0        143.00  143.00   321.750 -178.750       648   
2           0.0        143.00  143.00   321.750 -1

In [127]:
colour_indicators_df.dtypes

create_dt                   object
metric_id                   object
feature_name                object
metrics                     object
display_names               object
operating_system_type       object
1st Quartile               float64
3rd Quartile               float64
IQR                        float64
Upper                      float64
Lower                      float64
Yesterday                   object
last_7_days                float64
last_30_days               float64
% Change Last 7 Days        object
% Change Last 30 Days       object
metric_nature_indicator     object
colour_indicator            object
dtype: object


In [130]:
import pandas as pd

# Convert 'Yesterday' to numeric (float) type in Pandas
colour_indicators_df['Yesterday'] = pd.to_numeric(colour_indicators_df['Yesterday'], errors='coerce')
colour_indicators_df['% Change Last 7 Days'] = pd.to_numeric(colour_indicators_df['% Change Last 7 Days'], errors='coerce')
colour_indicators_df['% Change Last 30 Days'] = pd.to_numeric(colour_indicators_df['% Change Last 30 Days'], errors='coerce')
# Check the dtypes to verify the change
print(colour_indicators_df.dtypes)


create_dt                   object
metric_id                   object
feature_name                object
metrics                     object
display_names               object
operating_system_type       object
1st Quartile               float64
3rd Quartile               float64
IQR                        float64
Upper                      float64
Lower                      float64
Yesterday                  float64
last_7_days                float64
last_30_days               float64
% Change Last 7 Days       float64
% Change Last 30 Days      float64
metric_nature_indicator     object
colour_indicator            object
dtype: object
<stdin>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<stdin>:5: SettingWithCopyWarning: 
A value is trying to 

In [131]:
fourth_table_spark_df = spark.createDataFrame(colour_indicators_df)

In [132]:
fourth_table_spark_df.printSchema()

root
 |-- create_dt: date (nullable = true)
 |-- metric_id: string (nullable = true)
 |-- feature_name: string (nullable = true)
 |-- metrics: string (nullable = true)
 |-- display_names: string (nullable = true)
 |-- operating_system_type: string (nullable = true)
 |-- 1st Quartile: double (nullable = true)
 |-- 3rd Quartile: double (nullable = true)
 |-- IQR: double (nullable = true)
 |-- Upper: double (nullable = true)
 |-- Lower: double (nullable = true)
 |-- Yesterday: double (nullable = true)
 |-- last_7_days: double (nullable = true)
 |-- last_30_days: double (nullable = true)
 |-- % Change Last 7 Days: double (nullable = true)
 |-- % Change Last 30 Days: double (nullable = true)
 |-- metric_nature_indicator: string (nullable = true)
 |-- colour_indicator: string (nullable = true)


In [134]:
from pyspark.sql.types import IntegerType

# Convert 'metric_id' from string to integer type
fourth_table_spark_df_transformed = fourth_table_spark_df.withColumn("metric_id", fourth_table_spark_df["metric_id"].cast(IntegerType()))

# Show the first few rows to verify the data
fourth_table_spark_df_transformed.show()


+----------+---------+-------------+--------------------+--------------------+---------------------+------------+------------+------+--------+--------+---------+-----------+------------+--------------------+---------------------+-----------------------+----------------+
| create_dt|metric_id| feature_name|             metrics|       display_names|operating_system_type|1st Quartile|3rd Quartile|   IQR|   Upper|   Lower|Yesterday|last_7_days|last_30_days|% Change Last 7 Days|% Change Last 30 Days|metric_nature_indicator|colour_indicator|
+----------+---------+-------------+--------------------+--------------------+---------------------+------------+------------+------+--------+--------+---------+-----------+------------+--------------------+---------------------+-----------------------+----------------+
|2025-02-10|       22|   COX MOBILE|DR_Mobile Lock En...|                    |            Apple iOS|         0.0|       143.0| 143.0|  321.75| -178.75|     42.0|     50.714|      45.833| 

In [135]:
# Verify the schema to make sure the change was applied
fourth_table_spark_df_transformed.printSchema()

root
 |-- create_dt: date (nullable = true)
 |-- metric_id: integer (nullable = true)
 |-- feature_name: string (nullable = true)
 |-- metrics: string (nullable = true)
 |-- display_names: string (nullable = true)
 |-- operating_system_type: string (nullable = true)
 |-- 1st Quartile: double (nullable = true)
 |-- 3rd Quartile: double (nullable = true)
 |-- IQR: double (nullable = true)
 |-- Upper: double (nullable = true)
 |-- Lower: double (nullable = true)
 |-- Yesterday: double (nullable = true)
 |-- last_7_days: double (nullable = true)
 |-- last_30_days: double (nullable = true)
 |-- % Change Last 7 Days: double (nullable = true)
 |-- % Change Last 30 Days: double (nullable = true)
 |-- metric_nature_indicator: string (nullable = true)
 |-- colour_indicator: string (nullable = true)


In [136]:
fourth_output_path = "s3://cci-dig-aicoe-data-sb/processed/healthscore/healthscore_calculations/"

In [138]:
fourth_df_table_write_df = fourth_table_spark_df_transformed.coalesce(1).write.format("parquet").mode("append").save(fourth_output_path)

In [96]:
#colour_indicators_df.to_csv(r'D:\Healthsore Data\Table4_athena.csv', index = False)
